# Cash flow modelling


## Date: August 2024


### For illustration and educational purposes only.

Simple cash flow modelling based around UK.

### Assumptions:

1) UK State Pension is assumed to increase by the rate of inflation rate.

https://www.gov.uk/check-state-pension

2) UK State Pension age is 67

3) UK State Pension is £11502 in 2024

3) Reqired income is assumed to increase by the rate of inflation

4) Fixed assets e.g. house, are assumed to increase by the rate of inflation

5) Savings increase by a fixed market rate 

6) Defined Contribution Pensions increase by a fixed market rate

7) Defined Benefits increase by the rate of inflation

8) Further contributions to DC pensions during employment are not included i.e. DC pensions are static and grow only through investment returns at the market rate.

8) Employment income is assumed to increase at the rate of inflation

### Notes:

1 - Fidelity and Towers Watson assume a rate of inflation of 2.5 % for their modelling

2 - IFA modelling assumed 3% for RPI and 2% for CPI

4 - The Pension and Lifetime Savings Association (PLSA) estimates a single person who owns their own home will need an income of more than £43.000 for comfortable retirement, £31.300 for a moderate retirement and £14.4 for a minimum retirement.

##### Single
| Minimum | Moderate | Comfortable |
| ---     | ---      | ---         |
| 14.4    | 31.3     | 43.1        |

##### Couple
| Minimum | Moderate | Comfortable |
| ---     | ---      | ---         |
| 22.4    | 43.1     | 59.0        |

[https://www.retirementlivingstandards.org.uk/]

5 - UK Minimum Living Wage in 2024 is 20.82

6 - UK Average Wage in 2024 is 35.464

7 - Average pension pots by age UK (December 2023)

| Age | Average Pension Pot £k|
| --- | ------------------- |
|25-34|   9.300             |
|35-44|  30.000             |
|45-54|  75.000             |
|55-64| 107.300             |

[https://www.nutmeg.com/nutmegonomics/will-your-pension-be-big-enough-for-you-to-retire-at-55]


### Office for National Statistics Life Expectancy Calculator

The ONS provides an interactive calculator to calculate life expectancy.

Provide current age and gender in the calculator to estimate life expectancy

https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies/articles/lifeexpectancycalculator/2019-06-07


In [ ]:
from IPython.display import IFrame
    
IFrame("https://www.ons.gov.uk/visualisations/dvc221/index.html", width=1000, height=800)


### UK State Pension


    

In [ ]:
import numpy             as np
import pandas            as pd
import openpyxl

import matplotlib.pyplot as plt
import matplotlib.dates  as mdates

from   IPython.display        import display, Markdown, Latex
from   datetime               import datetime,timedelta
from   dateutil.relativedelta import relativedelta

DaysPerMonth = 365./12.

def EstimateFuturePension(DatetimeBirthday,
                          RetirementAge         =   67,
                          DatetimeToday         = None,
                          PensionToday          =    1,
                          AnnualRateOfInflation =  3.5,
                          Debug                 = False):          
    
    '''
    EstimateFuturePension - Estimate pension value at a future date using annual inflation rate

    '''    
    
    retirementDate     = ( DatetimeBirthday + relativedelta(years = RetirementAge))
    
    monthsToRetirement = ( retirementDate - DatetimeToday ).days \
                        / ( DaysPerMonth )
    
    ForecastPension    = calculateCompoundInterest(StartValue          = PensionToday,
                                                   AnnualRateInPercent = AnnualRateOfInflation,
                                                   TimeInMonths        = monthsToRetirement)
    
    if Debug:
        
        print("INFO The inflation pension is ",ForecastPension)

    return ForecastPension


def monthToYear(x):
    return x * 12

def yearToMonth(x):
    return x / 12

class DefinedContribution:
    
    def __init__(self, Name, StartDate, AnnualReturn):
        
        self.Name                   = Name
        self.StartDateOfPension     = StartDate
        self.AnnualReturn           = AnnualReturn
        
        self.MonthsPerYear          = 12.0
        
class DefinedBenefit:
    
    def __init__(self, Name, StartDate, AnnualValue, AddInflation=True):
        
        self.Name                   = Name
        self.StartDateOfPension     = StartDate
        self.StartAnnualValue       = AnnualValue
        self.AddInflation           = AddInflation
                
        self.MonthsPerYear  = 12.0
        
    def GetMonthlyPayment(self, Date, AnnualRateOfInflation=2.5):
        
        MonthlyPayment = 0.0
                
        if Date > self.StartDateOfPension:
                
            MonthlyPayment =  self.StartAnnualValue        \
                            / self.MonthsPerYear
            
            if self.AddInflation:
                
                difference = relativedelta( Date ,self.StartDateOfPension )
                
                numberOfMonths =   ( difference.months                 ) \
                                 + ( difference.years  * self.MonthsPerYear )
                
                MonthlyPayment = calculateCompoundInterest     \
                                         (StartValue          = MonthlyPayment,
                                          AnnualRateInPercent = AnnualRateOfInflation,
                                          TimeInMonths        = numberOfMonths)
                
        return MonthlyPayment

def calculateCompoundInterest(StartValue=1,AnnualRateInPercent=2.5,TimeInMonths=1):

    '''
    Inputs
    ------
    StartValue:       Initial deposit
    AnnualRate:       Annual interest rate
    TimeInMonths:     Number of months
    
    Outputs
    -------
    Calculated compund interest over TimeInMonths months for an annual interest rate of AnnualRate
    '''
    
    monthsPerYear = 12.

    return StartValue*(1.0 + (0.01*AnnualRateInPercent/monthsPerYear))**(TimeInMonths)

def calculateDefinedBenefits(DefinedBenefitPensions,
                             DateTime,
                             AnnualRateOfInflation):
                
    Contributions  = [DateTime]
    Total          = 0.0
    
    if isinstance(DefinedBenefitPensions, list):

        for _db in DefinedBenefitPensions:

            Contributions.append( _db.Name )
            Contributions.append( _db.GetMonthlyPayment(DateTime,
                                                        AnnualRateOfInflation=AnnualRateOfInflation) )
            Total += Contributions[-1]

    return [DateTime,Total],Contributions

def RunSimulation(SimulationName               =  None,
                  AnnualReturnRate             =   3.0,
                  AnnualRateOfInflation        =   2.0,
                  DefinedBenefitPensions       =  None,
                  PensionsDefinedContributions = 100.0,
                  InvestmentSavings            = 100.0,
                  FixedAssets                  =   0.0,
                  EmploymentIncome             =   0.0,
                  AnnualRequiredIncome         =  25.0,
                  DatetimeToday                =  None,
                  DatetimeBirthday             =  None,
                  DatetimeRetirement           =  None,
                  DatetimeDeath                =  None,
                  OtherDates                   =  None,
                  OutputName                   =  None,
                  Plot                         =  True,
                  Debug                        = False):
    
    '''
    
    '''
    
    monthsPerYear                = 12
                                                                                      # Initialise history 

    total,payments               = calculateDefinedBenefits(DefinedBenefitPensions,
                                                            DatetimeToday,
                                                            AnnualRateOfInflation)
        
    pensionsDefinedBenefit       = [ total                                                   ] # Total of all Defined Benefits
    
    definedBenefitsIncome        = [ payments                                                ]  # Separate DB pensions
        
    investmentsSavings           = [ [ DatetimeToday, InvestmentSavings                    ] ]  # Non pension savings

    pensionsDefinedContributions = [ [ DatetimeToday, PensionsDefinedContributions         ] ]  # AKA Personal Money Pot
                                                                                    
    shortFall                    = [ [ DatetimeToday, 0.0                                  ] ]
    
    fixedAssets                  = [ [ DatetimeToday, FixedAssets                          ] ]

    employmentIncome             = [ [ DatetimeToday, EmploymentIncome     / monthsPerYear ] ]  # Employment income
        
    requiredBudget               = [ [ DatetimeToday, AnnualRequiredIncome / monthsPerYear ] ]  # Required Annual Budget 
        
    t                            = DatetimeToday
    
    complete                     = False

    while not complete:                                        

        t  += relativedelta(months = 1)                         # Monthly increments to model

        age = relativedelta(t,DatetimeBirthday)

        if t > DatetimeDeath:                                   # Complete simulation

            complete = True

            break

        # Step 1 - Update Savings by rate of return-----------------------------
            
        investmentsSavings          .append( [t,calculateCompoundInterest(investmentsSavings[-1][1],
                                                                          AnnualRateInPercent= AnnualReturnRate,
                                                                          TimeInMonths       = 1 )] )

        pensionsDefinedContributions.append( [t,calculateCompoundInterest(pensionsDefinedContributions[-1][1],
                                                                          AnnualRateInPercent = AnnualReturnRate,
                                                                          TimeInMonths        = 1 )] )
                    
        # Step 2 - Update Defined Benefit Pensions by rate of inflation-----------------------------

        total,payments = calculateDefinedBenefits(DefinedBenefitPensions,
                                                  t,
                                                  AnnualRateOfInflation)

        
        definedBenefitsIncome .append ( payments )

        pensionsDefinedBenefit.append ( total    )

        # Step 3 - Update Required income by rate of inflation-----------------------------
                
        requiredIncome     = calculateCompoundInterest(requiredBudget[-1][1],     # Add inflation to the required income required
                                                       AnnualRateInPercent = AnnualRateOfInflation,
                                                       TimeInMonths        = 1)

        if t < DatetimeRetirement:                                       # Before retirement the income required from pension is 0

            requiredBudget.append( [t,  requiredIncome   ])

        else :                                                           # After retirement so add in the required income

            requiredBudget.append( [ t, requiredIncome ] )
            
        # Step 4 - Update employment income by rate of inflation-----------------------------
                
        income     = calculateCompoundInterest(employmentIncome[-1][1],     # Add inflation to the employment income required
                                               AnnualRateInPercent = AnnualRateOfInflation,
                                               TimeInMonths        = 1)

        if t < DatetimeRetirement:                                       # Before retirement there is employment income

            employmentIncome.append( [t,income])

        else :                                                           # After retirement employment income is 0

            employmentIncome.append( [ t, 0   ] )


        # Step 5 - Update fixed assets by rate of inflation
        
        fixedAssets  .append( [t,calculateCompoundInterest( fixedAssets[-1][1],
                                                            AnnualRateInPercent = AnnualRateOfInflation,
                                                            TimeInMonths        = 1 )] )

            
        # Step 7 - Calculate the short fall i.e. difference between required income and income
        #          from defined benefits and employment -----------------------------------------------------
            
        if t < DatetimeRetirement:                                       # Pre retirement so only employment income

            shortFall.append( [t,  requiredBudget         [-1][1]
                                 - employmentIncome       [-1][1]] )

        else:                                                            # Post retirement so Pensions start
                                                                         # but no employment income
            shortFall.append( [t,  requiredBudget         [-1][1]
                                 - pensionsDefinedBenefit [-1][1]] )

        # Step 8 - Calculate the drawdown required from savings and Defined Contributions
        #          needed to fund the shortfall------------------------------------------

        if ( shortFall[-1][1] > 0 ):                                    # There is a shortfall that needs to be funded
        
            if      ( t < DatetimeRetirement ):                         # Shortfall before retirement 
                                                                        # funded from savings ONLY

                investmentsSavings[-1][1]            -= shortFall[-1][1]
                
            else:                                                       # Shortfall after retirement
                                                                        # so proportionately withdraw  
                                                                        # monies from savings AND  
                                                                        # defined contribution funds

                totalOfSavingsAndDefinedContributions =  pensionsDefinedContributions[-1][1] \
                                                       + investmentsSavings          [-1][1] 

                investmentsSavings[-1][1]            -= shortFall[-1][1] * (   investmentsSavings          [-1][1] \
                                                                             / totalOfSavingsAndDefinedContributions )

                pensionsDefinedContributions[-1][1]  -= shortFall[-1][1] * (   pensionsDefinedContributions[-1][1] \
                                                                             / totalOfSavingsAndDefinedContributions )
                
        if Debug:

            print (t.strftime("%m/%Y"),
                   age.years,
                   np.around(shortFall             [-1][1],2),
                   np.around(pensionsDefinedBenefit[-1][1],2))

    # Simulation completed

    investmentsSavings           = np.array( investmentsSavings           )
    pensionsDefinedContributions = np.array( pensionsDefinedContributions )
    pensionsDefinedBenefit       = np.array( pensionsDefinedBenefit       )
    requiredBudget               = np.array( requiredBudget               )
    employmentIncome             = np.array( employmentIncome             )
    shortFall                    = np.array( shortFall                    )
    fixedAssets                  = np.array( fixedAssets                  )
    definedBenefitsIncome        = np.array( definedBenefitsIncome        )
    
    Results =  {"InvestmentSavingsHistory" : investmentsSavings,
                "PensionsDCHistory"        : pensionsDefinedContributions,
                "PensionsDBHistory"        : pensionsDefinedBenefit,
                "RequiredBudgetHistory"    : requiredBudget,
                "ShortfallHistory"         : shortFall,
                "DBIncomeHistory"          : definedBenefitsIncome,
                "FixedAssetsHistory"       : fixedAssets,
                "EmploymentIncome"         : employmentIncome}
    
    if Plot:
        
        totalSavings = pensionsDefinedContributions[:, 1]+investmentsSavings[:,1] 
        totalSavings = totalSavings.astype('float')

        plt.figure(figsize=(16,22))

        timeMargin = (5*monthsPerYear) * np.diff(shortFall [:, 0])[0]

        plt.subplot(3,1,1)                                                              # Upper plot

        plt.fill_between(pensionsDefinedContributions[:, 0],totalSavings,y2=0,
                         where=totalSavings>0,
                         color='gray',
                         alpha=0.4)

        plt.fill_between(pensionsDefinedContributions[:, 0],totalSavings,y2=0,
                         where=totalSavings<0,
                         color='red',
                         alpha=0.4)

        for datetimes,ts,text,styl in [[pensionsDefinedContributions[:, 0],pensionsDefinedContributions[:, 1],"Pension DC","g--"],
                                       [investmentsSavings          [:, 0],investmentsSavings          [:,1],"Savings","b:"],
                                       [fixedAssets                 [:, 0],fixedAssets                 [:,1],"Fixed","c:"  ],
                                       [pensionsDefinedContributions[:, 0],totalSavings                     ,"Total","k-"  ]]:

            plt.plot(datetimes    ,ts, styl,linewidth=2)
            plt.text(datetimes[-1],ts[-1],text,
                     verticalalignment  ='center',
                     horizontalalignment='left' )
            plt.text(datetimes[ 0],ts[ 0],text,
                     verticalalignment  ='center',
                     horizontalalignment='right')

        plt.ylim(min(0,1.1*np.min(totalSavings)),1.1*np.max(totalSavings))
        plt.xlim(np.min(shortFall[:,0])-timeMargin,np.max(shortFall[:,0])+timeMargin)

        yLower,yUpper = plt.ylim()
        xLower,xUpper = plt.xlim()

        for a in range(ageToday,statisticalAge):
            plt.text(DatetimeBirthday+relativedelta(years=a),yLower,
                     str(a),
                     horizontalalignment='center')

        if OtherDates is not None:
            
            for otherDates in OtherDates:
                plt.text(    otherDates[0],yUpper,
                         str(otherDates[1]),
                         horizontalalignment='left',
                         rotation=45)
            
        text = "Return Rate:    "+str(AnnualReturnRate     )+"% pa \n"  \
              +"Inflation Rate: "+str(AnnualRateOfInflation)+"% pa"
        
        plt.text(1.01*xUpper,0.75*(yLower+yUpper),
                 text,
                 fontsize = 10,
                 bbox     = dict(facecolor='white', alpha=0.5))
            
        for values in [ [DatetimeToday          ,"Today"                ],
                        [DatetimeRetirement     ,"Retirement"           ],
                        [DatetimeDeath          ,"Life Expectancy"      ] ]:

            plt.plot( [values[0]]*2,[yLower,yUpper],
                     'k--',alpha=0.5,linewidth=1)
            plt.text(values[0],yUpper,values[1],
                     horizontalalignment='left',rotation=45)

        for _db in DefinedBenefitPensions:

            plt.plot( [_db.StartDateOfPension]*2,
                      [yLower,yUpper],
                     'k--',alpha=0.5,linewidth=2)
            plt.text(_db.StartDateOfPension,yUpper,_db.Name,
                     horizontalalignment='left',rotation=45)
            
        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.gca().xaxis.set_major_locator  (mdates.YearLocator())
        plt.gcf().autofmt_xdate()
        
        plt.ylabel("Asset      \nValue      \n[£k]      ",
                   rotation=0,
                   horizontalalignment='right')
        plt.grid()
        
        plt.title("Assets",loc='left',fontsize=16)

        plt.subplot(3,1,2)                                     # Lower PLot

        for c in range(2,definedBenefitsIncome.shape[1],2):
        
            name = definedBenefitsIncome[0,c-1]
            plt.plot(definedBenefitsIncome[:, 0],definedBenefitsIncome[:, c],':',label=name,linewidth=2)
            plt.text(definedBenefitsIncome[-1,0],definedBenefitsIncome[-1,c],name)
        
        plt.legend(loc='upper left')
            
        plt.plot(pensionsDefinedBenefit[ :,0],pensionsDefinedBenefit[ :,1],linewidth=2,alpha=0.6)
        plt.text(pensionsDefinedBenefit[-1,0],pensionsDefinedBenefit[-1,1],"Total DB")

        yLower,yUpper = plt.ylim()

        for a in range(ageToday,statisticalAge):
            plt.text(datetimeBirthday+relativedelta(years=a),yLower,str(a),horizontalalignment='center')

        for values in [ [DatetimeToday          ,"Today"                ],           # Events
                        [DatetimeRetirement     ,"Retirement"           ],
                        [DatetimeDeath          ,"Life Expectation"     ] ]:

            plt.plot( [values[0],values[0]],
                      [yLower   ,yUpper   ],
                     'k--',
                     alpha    = 0.5,
                     linewidth = 1)
            plt.text(values[0],yUpper,values[1],
                     horizontalalignment='left',rotation=45)

        for _db in DefinedBenefitPensions:

            plt.plot( [_db.StartDateOfPension]*2,
                      [yLower,yUpper],
                     'k--',
                     alpha=0.5,
                     linewidth=1)
            plt.text(_db.StartDateOfPension,yUpper,_db.Name,
                     horizontalalignment='left',rotation=45)
           
        plt.ylim(yLower,yUpper)
        plt.xlim(np.min(shortFall[:,0])-timeMargin,np.max(shortFall[:,0])+timeMargin)

        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.gca().xaxis.set_major_locator(mdates.YearLocator())
        plt.gcf().autofmt_xdate()

        secax = plt.gca().secondary_yaxis('right', functions=(monthToYear,yearToMonth))
        secax .set_ylabel("Annual            \nValues [£k]             ",
                          rotation = 0,
                          horizontalalignment='left')
        
        yticks = np.arange(10*((yLower*12)//10),10*(((10+yUpper)*12)//10),10)
        secax.set_yticks(yticks)
        
        xLower,xUpper = plt.xlim()
        for yt in yticks:
            plt.plot([xLower,xUpper],[yearToMonth(yt),yearToMonth(yt)],'k:',
                     alpha=0.5,
                     linewidth=1)
        
        plt.xlabel("Years")
        plt.ylabel("Monthly            \nValues [£k]             ",rotation=0)

        if SimulationName is None:
            
            plt.suptitle("Simulation Results",fontsize=36)

        else:
            
            plt.suptitle(SimulationName,fontsize=36)
            
        plt.grid()
        
        plt.title('DB Pensions',loc='left',fontsize=16)
        
        plt.subplot(3,1,3)                                     # Lower PLot

        plt.plot(requiredBudget       [:, 0],requiredBudget       [: , 1],'r-',alpha=0.5,linewidth=2)
        plt.text(requiredBudget       [-1,0],requiredBudget       [-1, 1],"RequiredBudget")

        plt.plot(employmentIncome     [:, 0],employmentIncome     [: , 1],'b-',alpha=0.5,linewidth=2)
        plt.text(employmentIncome     [-1,0],employmentIncome     [-1, 1],"Employment")

        plt.plot(shortFall            [ :,0],shortFall            [: , 1],'k-',alpha=0.5,linewidth=2)
        plt.text(shortFall            [-1,0],shortFall            [-1, 1],"ShortFall")

        yLower,yUpper = plt.ylim()

        for a in range(ageToday,statisticalAge):
            plt.text(datetimeBirthday+relativedelta(years=a),yLower,str(a),horizontalalignment='center')

        for values in [ [DatetimeToday          ,"Today"                ],           # Events
#                        [DatetimeRetirement     ,"Retirement"           ],
                        [DatetimeDeath          ,"Life Expectation"     ] ]:

            plt.plot( [values[0],values[0]],
                      [yLower   ,yUpper   ],
                     'k--',
                     alpha=0.5,
                     linewidth=1)
            plt.text(values[0],yUpper,values[1],
                     horizontalalignment='left',rotation=45)

        for _db in DefinedBenefitPensions:

            plt.plot( [_db.StartDateOfPension]*2,
                      [yLower,yUpper],
                     'k--',
                     alpha=0.5,
                     linewidth=1)
            plt.text(_db.StartDateOfPension,yUpper,_db.Name,
                     horizontalalignment='left',rotation=45)
           
        plt.ylim(yLower,yUpper)
        plt.xlim(np.min(shortFall[:,0])-timeMargin,np.max(shortFall[:,0])+timeMargin)

        plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        plt.gca().xaxis.set_major_locator(mdates.YearLocator())
        plt.gcf().autofmt_xdate()

        secax = plt.gca().secondary_yaxis('right', functions=(monthToYear,yearToMonth))
        secax .set_ylabel("Annual            \nValues [£k]             ",
                          rotation = 0,
                          horizontalalignment='left')
        
        yticks = np.arange(10*((yLower*12)//10),10*(((10+yUpper)*12)//10),10)
        secax.set_yticks(yticks)
        
        xLower,xUpper = plt.xlim()
        for yt in yticks:
            plt.plot([xLower,xUpper],[yearToMonth(yt),yearToMonth(yt)],'k:',
                     alpha=0.5,
                     linewidth=1)
        
        plt.xlabel("Years")
        plt.ylabel("Monthly            \nValues [£k]             ",rotation=0)

        if SimulationName is None:
            
            plt.suptitle("Simulation Results",fontsize=36)

        else:
            
            plt.suptitle(SimulationName,fontsize=36)
            
        plt.grid()
        
        plt.title("Totals",loc='left',fontsize=16)

        if OutputName is not None:
        
            plt.savefig(OutputName+".png", bbox_inches='tight')
            plt.close()
        
            if Debug:
                print("***INFO*** Saved plot to "+OutputName+".png")
                
        else:
            
            plt.show()
            plt.clf()
    
    if OutputName is not None:                                        # Write results to Excel file
    
        _dfP = pd.DataFrame(np.array \
                             ([["annualReturnRate"            ,AnnualReturnRate                            ],
                               ["annualRateOfInflation"       ,AnnualRateOfInflation                       ],
                               ["annualRequiredIncome"        ,AnnualRequiredIncome                        ],
                               ["investmentSavings"           ,InvestmentSavings                           ],
                               ["pensionsDefinedContributions",PensionsDefinedContributions                ],
                               ["DateToday"                   ,DatetimeToday          .strftime("%d/%m/%Y")],
                               ["DateBirthday"                ,DatetimeBirthday       .strftime("%d/%m/%Y")],
                               ["DateRetirement"              ,DatetimeRetirement     .strftime("%d/%m/%Y")],
                               ["DateDeath"                   ,DatetimeDeath          .strftime("%d/%m/%Y")]]),
                           columns=["SimulationParameter","Value"])


        _dfR = pd.DataFrame()
        
        _dfR['DateTime'] = [r.strftime("%d/%m/%Y") for r in Results['InvestmentSavingsHistory'][:,0]] 
        
        for k,v in Results.items():
            
            if k.find('DBIncomeHistory') < 0:                            # Exclude the DBIncomeHistory as it 
                                                                         # requires sepcial treatment
                _dfR[k] = v[:,1]

        for i in range(1,Results['DBIncomeHistory'].shape[1],2):         # Special Treatemnt for individual DB 

            name       ='IndividualDBIncomeHistory_' + Results['DBIncomeHistory'][0,i]
            _dfR[name] = Results['DBIncomeHistory'][:,i+1]
                
        _dfDb = pd.DataFrame(columns=["Name",
                                      "StartAnnualValue",
                                      "StartDateOfPension",
                                      "AddInflation"])
        
        _dfDb["Name"              ] = [_db.Name                                    for _db in definedBenefitPensions]
        _dfDb["StartAnnualValue"  ] = [_db.StartAnnualValue                        for _db in definedBenefitPensions]
        _dfDb["StartDateOfPension"] = [_db.StartDateOfPension.strftime("%d/%m/%Y") for _db in definedBenefitPensions]
        _dfDb["AddInflation"      ] = [_db.AddInflation                            for _db in definedBenefitPensions]
            
        print("***INFO*** Writing simulation results to output file: "+OutputName)
        
        with pd.ExcelWriter(OutputName) as writer:
            
            _dfP .to_excel(writer, sheet_name='Parameters'     ,index=False,header=False)
            _dfDb.to_excel(writer, sheet_name='DefinedBenefits',index=False)
            _dfR .to_excel(writer, sheet_name='Results'        ,index=False)
            
            _df = pd.DataFrame()
            _df.to_excel(writer, sheet_name='Plots')
            
            workbook  = writer.book
            worksheet = writer.sheets['Plots']

            worksheet.add_image(openpyxl.drawing.image.Image(OutputName+'.png'))
    
    return Results

# Test Scenario

In [ ]:
birthday                     = "01/01/1970"   # Set some useful defaults 

retirementAge                = 60
stateRetirementAge           = 67
statisticalAge               = 84

datetimeToday                = datetime.now()
datetimeBirthday             = datetime.strptime(birthday,"%d/%m/%Y") 
datetimeRetirement           = datetimeBirthday + relativedelta(years = retirementAge     )
datetimeStateRetirement      = datetimeBirthday + relativedelta(years = stateRetirementAge)
datetimeDeath                = datetimeBirthday + relativedelta(years = statisticalAge    )

minimumLivingWage            = 20.82
averageWage                  = 35.464

annualReturnRate             = 4.00
annualRateOfInflation        = 2.50

investmentSavings            = 250.0  # Banks, Building societies, stocks, ISAs etc
pensionsDefinedContributions = 200.0  # DC Pension savings

fixedAssets                  = 500   # e.g. House. Unused in calculations

definedBenefitPensions       = []

definedBenefitPensions.append( DefinedBenefit("DefBen1",
                               StartDate    = datetimeBirthday + relativedelta(years = 60),
                               AnnualValue  = 5.0,
                               AddInflation = True ) )

ukStatePensionToday          = 11.502

if True:                                                                     # Estimate UK state pension 
                                                                             # assuming it is maintained 
                                                                             # to keep up with inflation
    
    ukStatePensionForecast = EstimateFuturePension(datetimeBirthday,
                                                   stateRetirementAge,
                                                   datetimeToday,
                                                   ukStatePensionToday,
                                                   annualRateOfInflation)
    
    print("INFO The inflation adjusted UK State pension is ",ukStatePensionForecast)

    definedBenefitPensions.append( DefinedBenefit("State",
                                   StartDate    = datetimeBirthday + relativedelta(years = 67),
                                   AnnualValue  = ukStatePensionForecast,
                                   AddInflation = True ) )
    
employmentIncome             = 35.0                                                 # Set a default value for
                                                                                    # annual employment income

annualRequiredIncome         = 30.0                                                 # Set a default value for
                                                                                    # Annual income required

                                                                                    # Define significant dates

ageToday                     = relativedelta(datetimeToday,datetimeBirthday).years



print ("***INFO*** Mid case scenario: Average Rates of Return and Inflation")


results = RunSimulation                                                   \
             (AnnualReturnRate             = annualReturnRate,            \
              AnnualRateOfInflation        = annualRateOfInflation,       \
              DefinedBenefitPensions       = definedBenefitPensions,      \
              AnnualRequiredIncome         = annualRequiredIncome,        \
              EmploymentIncome             = employmentIncome,            \
              InvestmentSavings            = investmentSavings,           \
              PensionsDefinedContributions = pensionsDefinedContributions,\
              FixedAssets                  = fixedAssets,                 \
              DatetimeToday                = datetimeToday,               \
              DatetimeBirthday             = datetimeBirthday,            \
              DatetimeRetirement           = datetimeRetirement,          \
              DatetimeDeath                = datetimeDeath,               \
              OtherDates                   = None,                  \
              #OutputName                   = "./TestSimulationResults.xlsx",   \
              Plot                         = True,
              Debug                        = False)
